In [ ]:
import numpy as np
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from matplotlib import pylab as plt
import matplotlib.cm as cm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from scipy.stats import multivariate_normal
from scipy.special import logsumexp
import pandas as pd
%matplotlib inline

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
raw_kaggle_songs = pd.read_csv('labeled_songs.csv')
#spotify_songs = pd.read_csv('happy_songs(preprocess1).csv')
raw_kaggle_songs.shape

(15405, 27)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# happy_songs = kaggle_songs.loc[kaggle_songs['label'] == 'Joy'].sample(n=1000, random_state=1).reset_index(drop=True)
# split training/test set
kaggle_songs = raw_kaggle_songs.sample(n=7000, random_state=1).reset_index(drop=True)
test = pd.concat([raw_kaggle_songs, kaggle_songs]).drop_duplicates(keep=False).sample(n=7000, random_state=1).reset_index(drop=True)

# TODO: actually need to include all songs, add column isHappy and set 1 if yes, otherwise 0
kaggle_songs['isHappy'] = kaggle_songs.apply(lambda row: 1 if row['label'] == 'Joy' else 0, axis=1)
kaggle_songs['isSad'] = kaggle_songs.apply(lambda row: 1 if row['label'] == 'Sadness' else 0, axis=1)
test['isHappy'] = test.apply(lambda row: 1 if row['label'] == 'Joy' else 0, axis=1)
test['isSad'] = test.apply(lambda row: 1 if row['label'] == 'Sadness' else 0, axis=1)
# kaggle_songs.head(5)
test.head(5)

,Unnamed: 0,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language,label,isHappy,isSad
0,4422,1VFObW6RUUrKjWY0SahdMa,Are We Cool?,Mike Vallas,Are we cool? Is it all ok? Are we cool? Is it ...,37,0cc7f7pA20slPrte2Goc5I,Are We Cool?,2019-10-15,Electropop,2Z5cPJ6Z4EVZAfF08amjvL,pop,electropop,0.498,0.493,6,-8.428,1,0.0385,0.1680,0.000002,0.2150,0.293,180.066,188187,en,Sadness,0,1
1,1591,0q9YVW7oLuDqxfjGTxb7Lx,Lick It Up,KISS,"Yeah, yeah Don't want to wait until you know m...",0,3gbTIe7lJK7Xf33012sGIF,Discoveries,2008-10-20,Classic Hard Rock,1gAeQGQ7hr2q7IYmaejDW6,rock,hard rock,0.668,0.846,2,-4.149,1,0.0563,0.1090,0.000032,0.3200,0.529,120.276,236013,en,Joy,1,0
2,13399,5QIKZWOHDPbGh5EpSqCXMG,All the Things,Gaullin,All the things she said All the things she sai...,65,7fhzGoeqXEdVWVdxymGBHJ,All the Things,2018-02-07,🔊BASSBOOSTED🔊⚡ELECTRO HOUSE⚡🔥EDM CAR MUSIC2018...,4GSiiL8tcMgvoV7K1IADb8,edm,electro house,0.776,0.634,7,-10.345,1,0.1060,0.0102,0.168000,0.1190,0.475,125.043,174745,en,Joy,1,0
3,4722,1zK7i2QyuCmpYE209BMcPZ,Son And Daughter - Remastered 2011,Queen,"I want you, ahh Woman Tried to be a son and da...",34,1kkb8xlG9yssEVsWKiEtAB,Queen (2011 Remaster),1973-07-13,The Queen - La Discografia Completa,3E88dLx4fgFYY70gdGzdnB,rock,album rock,0.328,0.723,2,-7.842,1,0.0434,0.1300,0.085700,0.3690,0.749,144.832,199533,en,Joy,1,0
4,17104,7CMAwm7mEkZkkaBl82DicE,2 Of Amerikaz Most Wanted - (Explicit),2Pac,Up out of there Ain't nothin' but a gangsta pa...,1,3oaGekoJ37K0waoYjDzaRr,The Best of 2Pac - Pt. 1: Thug,2007-12-04,Gangsta Rap/90's Hip-Hop,62spXXfUxBed8nbd5xvH2O,rap,gangster rap,0.710,0.905,1,-2.578,1,0.3240,0.0391,0.000930,0.0562,0.817,198.500,246147,en,Fear,0,0


PREPROCESSING

In [ ]:
# PREPROCESSING
# happy_songs = happy_songs.drop_duplicates()
kaggle_songs = kaggle_songs.drop_duplicates()

def lemmatize(text):
  lemmatizer = WordNetLemmatizer()
  tokenization = nltk.word_tokenize(text)
  return ' '.join([lemmatizer.lemmatize(words) for words in tokenization ])

def strip_punct(text):
  table = str.maketrans(dict.fromkeys(string.punctuation))
  return text.translate(table)

def strip_stop(unwanted_word_list, text):
  return ' '.join(['' if word in unwanted_word_list else word for word in text.split()])

#convert to lowercase
kaggle_songs["lyrics"] = kaggle_songs["lyrics"].str.lower()
test["lyrics"] = test["lyrics"].str.lower()

# Strip punctuation
kaggle_songs["lyrics"] = kaggle_songs["lyrics"].apply(lambda x: strip_punct(x))
test["lyrics"] = test["lyrics"].apply(lambda x: strip_punct(x))

# # Lemmatize all the words
stemmer = WordNetLemmatizer()
kaggle_songs["lyrics"] = kaggle_songs["lyrics"].apply(lambda x: lemmatize(x))
test["lyrics"] = test["lyrics"].apply(lambda x: lemmatize(x))

# # Strip stop words
stop_words = stopwords.words('english')
kaggle_songs["lyrics"] = kaggle_songs["lyrics"].apply(lambda x: strip_stop(stop_words, x))
test["lyrics"] = test["lyrics"].apply(lambda x: strip_stop(stop_words, x))

# import nltk 
# #nltk.download()
# from nltk.stem import WordNetLemmatizer

# lemmatizer = WordNetLemmatizer()

# lemmWords = []
# lemmWords2 = []

# for i, row in trainingSet.iterrows():
#     tokenization = nltk.word_tokenize(row["text"])
#     lemmatized_string = ' '.join([lemmatizer.lemmatize(words) for words in tokenization ])
#     lemmWords.append(lemmatized_string)
    
# for i, row in developmentSet.iterrows():
#     tokenization = nltk.word_tokenize(row["text"])
#     lemmatized_string = ' '.join([lemmatizer.lemmatize(words) for words in tokenization ])
#     lemmWords2.append(lemmatized_string)


# trainingSet["text"] = lemmWords
# developmentSet["text"] = lemmWords2

#remove stop words 
# from nltk.corpus import stopwords
# stop = stopwords.words('english')

# trainingSet["text"] = trainingSet["text"].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
# developmentSet["text"] = developmentSet["text"].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))


In [ ]:
kaggle_songs['lyrics'][0]

'time fly    night  young daylight shine   undisclosed location location bloodshot eye looking   sun paradise  lived    called   vacation vacation youre painting   dream   wan na belong  wan na belong    hill  far away  million mile  la  anywhere away    know weve got  get away someplace   one know  name well find  start  something new  take  anywhere take  anywhere anywhere away    take  anywhere anywhere take  anywhere anywhere away   fun little le fun little le      mememe  mememe     oh fun little le fun little le      mememe  mememe     truth come    blacking  looking  connection   crowd  empty face empty face  secret    thing im craving   good   bad let   cause  could take   could take  youre painting   dream   wan na belong  wan na belong  wan na belong    hill  far away  million mile  la  anywhere away    know weve got  get away someplace   one know  name well find  start  something new  take  anywhere anywhere take  anywhere anywhere away   fun little le fun little le      mem

Bag of Words Model

In [ ]:
vectorizer = CountVectorizer(binary=True, min_df=5)
X_kaggle = vectorizer.fit_transform(kaggle_songs['lyrics'])

print('Size of Vocabulary/Features:', X_kaggle.shape[1])
print('vectorizer:', vectorizer.get_feature_names())

Size of Vocabulary/Features: 8077
vectorizer: ['03', '10', '100', '101', '11', '110', '12', '123', '13', '14', '15', '150', '16', '17', '18', '180', '187', '19', '1st', '20', '200', '2000', '2019', '21', '22', '23', '24', '247', '25', '27', '29', '2pac', '2x', '30', '300', '305', '36', '360', '38', '3d', '3rd', '3x', '40', '400', '42', '44', '45', '45s', '4th', '4x', '50', '500', '5th', '60', '62', '64', '69', '70', '711', '7th', '80', '800', '808', '84', '85', '88', '8ball', '90', '911', '92', '93', '94', '95', '96', '98', '99', 'aa', 'aah', 'aaliyah', 'aap', 'aaron', 'abandon', 'abandoned', 'abc', 'abide', 'ability', 'ablaze', 'able', 'aboard', 'absence', 'absolute', 'abuse', 'abused', 'abusing', 'abyss', 'ac', 'accent', 'accept', 'accepted', 'accident', 'account', 'accountant', 'accused', 'ace', 'ache', 'achieve', 'aching', 'acid', 'acknowledge', 'acre', 'acrobat', 'across', 'act', 'actin', 'acting', 'action', 'active', 'activity', 'actor', 'actress', 'actually', 'ad', 'adam', 'add'

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# ngram model?
ngram_vectorizer = CountVectorizer(binary=True, min_df=10, ngram_range=(1,3))
X_kaggle_ngram = ngram_vectorizer.fit_transform(kaggle_songs['lyrics'])
#happy 
X_kaggle_ngram.toarray()
happy_model = LogisticRegression(penalty='l2').fit(X_kaggle_ngram, kaggle_songs['isHappy'])
y_train_pred = happy_model.predict(X_kaggle_ngram)
train_f1_score = f1_score(kaggle_songs['isHappy'], y_train_pred)
train_f1_score
# Test happy 
X_test_kaggle_ngram = ngram_vectorizer.transform(test['lyrics'])
y_test_pred_ngram = happy_model.predict(X_test_kaggle_ngram)
test_ngram_f1_score = f1_score(test['isHappy'], y_test_pred_ngram)
test_ngram_f1_score

0.6460587326120557

In [ ]:
#happy 
X_kaggle.toarray()
happy_model = LogisticRegression(penalty='l2').fit(X_kaggle, kaggle_songs['isHappy'])
y_train_pred = happy_model.predict(X_kaggle)
train_f1_score = f1_score(kaggle_songs['isHappy'], y_train_pred)
train_f1_score

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.9823168531216168

In [ ]:
#sad 
X_kaggle.toarray()
sad_model = LogisticRegression(penalty='l2').fit(X_kaggle, kaggle_songs['isSad'])
y_train_pred = sad_model.predict(X_kaggle)
train_f1_score = f1_score(kaggle_songs['isSad'], y_train_pred)
train_f1_score

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.97843607057286

In [ ]:
test['lyrics']

0         cool    ok   cool    ok  said  dont want sta...
1       yeah yeah dont want  wait   know  better let  ...
2         thing  said   thing  said runnin   head runn...
3        want  ahh woman tried    son  daughter rolled...
4           aint nothin   gangsta party eh light   sno...
                              ...                        
6995    falling like domino reaching   star one  one h...
6996    got   car heading  coast left  worry  plan beh...
6997    well  dont know   came  tonight  got  feeling ...
6998     got ta get  together cuz  got someplace  go  ...
6999    ooh ooh something   dont know feel  creeping  ...
Name: lyrics, Length: 7000, dtype: object

In [ ]:
# Test happy 
X_test_kaggle = vectorizer.transform(test['lyrics'])
y_test_pred = happy_model.predict(X_test_kaggle)
test_f1_score = f1_score(test['isHappy'], y_test_pred)
test_f1_score

0.6661704503163379

In [ ]:
test['isHappy_pred'] = y_test_pred

In [ ]:
# Test sad
X_test_kaggle = vectorizer.transform(test['lyrics'])
y_test_pred = sad_model.predict(X_test_kaggle)
test_f1_score = f1_score(test['isSad'], y_test_pred)
test_f1_score

0.7006681514476615

In [ ]:
test['isSad_pred'] = y_test_pred

In [ ]:
test

,Unnamed: 0,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language,label,isHappy,isSad,isHappy_pred,isSad_pred
0,4422,1VFObW6RUUrKjWY0SahdMa,Are We Cool?,Mike Vallas,cool ok cool ok said dont want sta...,37,0cc7f7pA20slPrte2Goc5I,Are We Cool?,2019-10-15,Electropop,2Z5cPJ6Z4EVZAfF08amjvL,pop,electropop,0.498,0.493,6,-8.428,1,0.0385,0.168000,0.000002,0.2150,0.2930,180.066,188187,en,Sadness,0,1,0,1
1,1591,0q9YVW7oLuDqxfjGTxb7Lx,Lick It Up,KISS,yeah yeah dont want wait know better let ...,0,3gbTIe7lJK7Xf33012sGIF,Discoveries,2008-10-20,Classic Hard Rock,1gAeQGQ7hr2q7IYmaejDW6,rock,hard rock,0.668,0.846,2,-4.149,1,0.0563,0.109000,0.000032,0.3200,0.5290,120.276,236013,en,Joy,1,0,1,0
2,13399,5QIKZWOHDPbGh5EpSqCXMG,All the Things,Gaullin,thing said thing said runnin head runn...,65,7fhzGoeqXEdVWVdxymGBHJ,All the Things,2018-02-07,🔊BASSBOOSTED🔊⚡ELECTRO HOUSE⚡🔥EDM CAR MUSIC2018...,4GSiiL8tcMgvoV7K1IADb8,edm,electro house,0.776,0.634,7,-10.345,1,0.1060,0.010200,0.168000,0.1190,0.4750,125.043,174745,en,Joy,1,0,1,0
3,4722,1zK7i2QyuCmpYE209BMcPZ,Son And Daughter - Remastered 2011,Queen,want ahh woman tried son daughter rolled...,34,1kkb8xlG9yssEVsWKiEtAB,Queen (2011 Remaster),1973-07-13,The Queen - La Discografia Completa,3E88dLx4fgFYY70gdGzdnB,rock,album rock,0.328,0.723,2,-7.842,1,0.0434,0.130000,0.085700,0.3690,0.7490,144.832,199533,en,Joy,1,0,0,0
4,17104,7CMAwm7mEkZkkaBl82DicE,2 Of Amerikaz Most Wanted - (Explicit),2Pac,aint nothin gangsta party eh light sno...,1,3oaGekoJ37K0waoYjDzaRr,The Best of 2Pac - Pt. 1: Thug,2007-12-04,Gangsta Rap/90's Hip-Hop,62spXXfUxBed8nbd5xvH2O,rap,gangster rap,0.710,0.905,1,-2.578,1,0.3240,0.039100,0.000930,0.0562,0.8170,198.500,246147,en,Fear,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,9570,43jJTO4MBgmaRybAaIHVA3,Arcadia - Original Mix,Hardwell,falling like domino reaching star one one h...,22,78NkVaYps3XM1G6jr6jlOE,Arcadia,2014-07-21,big boom room — TOMORROWLAND EDC EDM BIG ROOM ...,2oalsaFUockoPf1DU3wrL8,edm,big room,0.650,0.915,6,-2.896,0,0.0321,0.000264,0.029300,0.1080,0.1300,128.007,298125,en,Joy,1,0,1,0
6996,295,088eGKZm5dgDxFauCEwOqp,Real Eyes,Fili,got car heading coast left worry plan beh...,38,77g3tHy63PwOI1IRxEtNMT,Real Eyes,2019-12-20,The Edge of Indie Poptimism,5vIdb67lQcdFwoVkE4UMep,pop,indie poptimism,0.455,0.510,1,-10.020,1,0.0488,0.523000,0.076100,0.0826,0.0368,100.145,254257,en,Joy,1,0,0,0
6997,9436,4090PyjhaukeGh0c93jh1Y,Stuck In The Middle With You,Stealers Wheel,well dont know came tonight got feeling ...,19,4neIU9gg7nkWZkNltoi4lq,Stuck In The Middle With You - The Hits Collec...,1998-01-01,Hard Rock Cafe Classics,3sv5ViKoPDNnZRsklzEGMN,rock,hard rock,0.749,0.740,7,-4.965,1,0.0334,0.055200,0.000569,0.1600,0.9620,124.463,206560,en,Sadness,0,1,0,1
6998,16514,6Zp0xvhkmV7OE1iKuvVclq,Heaven,Mary Mary,got ta get together cuz got someplace go ...,36,2gXi6y3T7xN3XW0jR4jCbl,Mary Mary,2005-11-29,Gospel,0pItJZBDZFvgAOdFnSITnJ,r&b,urban contemporary,0.564,0.839,1,-6.076,1,0.0766,0.008780,0.012100,0.3100,0.5240,94.629,225333,en,Fear,0,0,0,0


In [ ]:
happy_and_sad_songs = kaggle_songs[(kaggle_songs.label == 'Sadness') | (kaggle_songs.label == 'Joy')]
happy_and_sad_songs.groupby('isHappy').mean()

,Unnamed: 0,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,isSad
isHappy,,,,,,,,,,,,,,,
0,9291.747409,41.965889,0.612758,0.687118,5.418826,-6.851424,0.579447,0.086202,0.174359,0.038904,0.187022,0.498852,121.209971,233388.321244,1.0
1,9212.331437,40.039118,0.627679,0.677801,5.256046,-7.316755,0.599573,0.081568,0.181132,0.060768,0.177995,0.535943,120.071706,234539.855619,0.0


In [ ]:
from google.colab import files
test.to_csv('labled_songs_predictions.csv')
files.download('labled_songs_predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
##################################spotify songs#########################################

In [ ]:
#reading in dataset created from spotify 
spotify_songs = pd.read_csv('all_songs_1126_noNONE_allEN.csv')

#adding isSad to spotify data 
spotify_songs['isSad'] = spotify_songs.apply(lambda row: 1 if row['isHappy'] == 0 else 0, axis=1)

#test data
kaggle_data = raw_kaggle_songs.sample(n=6000, random_state=1).reset_index(drop=True)
kaggle_data['isHappy'] = kaggle_data.apply(lambda row: 1 if row['label'] == 'Joy' else 0, axis=1)
kaggle_data['isSad'] = kaggle_data.apply(lambda row: 1 if row['label'] == 'Sadness' else 0, axis=1)

In [ ]:
# Version 2 with audio features
kaggle_data_v2 = kaggle_data.copy()
x_train_audio_features = spotify_songs[['danceability', 'energy', 'loudness', 'valence']]
y_train_audio_features = spotify_songs['isHappy']
features_model = LogisticRegression(penalty='l2').fit(x_train_audio_features, y_train_audio_features)
preds = features_model.predict(kaggle_data_v2[['danceability', 'energy', 'loudness', 'valence']])
kaggle_data_v2['audio_features_preds'] = preds
kaggle_data_v2

,Unnamed: 0,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language,label,isHappy,isSad,audio_features_preds
0,8772,3rLZh34oJ1KzLkYwPVYUgP,Anywhere - Willy William Remix,Rita Ora,Time flies by when the night is young Daylight...,27,7JdaTcp2GIiBv3XSGLAe8W,Anywhere (Willy William Remix),2017-11-24,EDM TROPICAL,3pS63EDS40FVGYL41zAcU4,latin,tropical,0.715,0.556,11,-10.456,0,0.0323,0.00283,0.002210,0.1860,0.119,106.970,213084,en,Sadness,0,1,0
1,8821,3ScJy88F8KqGDfWu8XJhHx,If I Lose Myself - Alesso vs OneRepublic,OneRepublic,I stared up at the sun Thought of all of the p...,67,42UJjk8i8L0De7lQtu7sqi,Native,2014-01-01,Electropop Hits 2017-2020,7kyvBmlc1uSqsTL0EuNLrx,pop,electropop,0.519,0.754,2,-5.689,1,0.0402,0.25800,0.000004,0.1390,0.160,125.975,215187,en,Fear,0,0,0
2,1730,0RZXNlGPvfMo54fd6uCvAT,Knuck If You Buck (feat. Lil' Scrappy),Crime Mob,"Knuck if you buck, boy (Crime Mob, hoe) Knuck ...",60,09stXr7AeoB1PsE3RpMpyU,Crime Mob (U.S. PA Version),2004-07-13,Hip-Hop Drive,37i9dQZF1DWUFmyho2wkQU,rap,hip hop,0.865,0.844,8,-4.946,1,0.3110,0.05820,0.000000,0.3730,0.755,150.090,205120,en,Anger,0,0,1
3,7881,3fCqWsYPCPdLteUpeIOeyQ,Let's Wait Awhile,Janet Jackson,There's something I want to tell you There's s...,1,3W2rEBPIwXyogofa5b4e6x,Control,1986-02-04,New Jack Swing,3ykXidKLz1eYPvuGoFlD1e,r&b,new jack swing,0.592,0.188,1,-17.243,1,0.0377,0.82000,0.000000,0.1320,0.378,172.683,276689,en,Joy,1,0,0
4,7142,31QRCKy7JIwPBaQl9ArGBz,Who's Laughing Now,Jessie J,"Mummy, they call me names, they wouldn't let m...",49,0BZbTNqpXFg6lxNv78X7Lp,Who You Are (Platinum Edition),2011-01-01,post teen pop,6rjxP7GQKoqqgoakzxl3PY,pop,post-teen pop,0.784,0.842,1,-4.041,1,0.1870,0.00448,0.000029,0.1040,0.483,100.035,234880,en,Sadness,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,144,04g1aQFzaegB7ovWaMMIzz,Congratulations - Don Diablo VIP Mix,Don Diablo,Dreaming Dreaming about the days that we could...,51,4PFXME1mmhj54RxWMxH5UX,Congratulations (Don Diablo VIP Mix),2020-01-17,Electro House Top Tracks,1G0q0NK7g3C0XerNqq7GbL,edm,electro house,0.818,0.879,7,-4.677,0,0.2050,0.09120,0.005610,0.1400,0.256,123.980,195484,en,Sadness,0,1,1
5996,5971,2LBqCSwhJGcFQeTHMVGwy3,Die For You,The Weeknd,I'm findin' ways to articulate the feeling I'm...,67,4AdZV63ycxFLF6Hcol0QnB,Starboy,2016-11-24,Today's Hits (Clean),7ENISpOJhocpMJVcGb0qcT,r&b,hip pop,0.586,0.525,1,-7.163,0,0.0615,0.11100,0.000000,0.1340,0.508,133.629,260253,en,Sadness,0,1,0
5997,10551,4jJEQg6mr3fQsUCYTwhBEZ,Sun,Two Door Cinema Club,Ocean blue What have I done to you? Cut so dee...,60,6CPwf8Yo8a4cgBDVeShL5C,Beacon,2012-09-03,Indie/Jazz Poptimism,6OaTudLqBEuTyUMPxRNpZc,pop,indie poptimism,0.647,0.534,4,-5.439,0,0.0279,0.01100,0.000249,0.1040,0.673,96.980,187720,en,Sadness,0,1,1
5998,11187,4rFKp5rDJS3bldnga3QDLY,Everything and More (feat. Aaron Cole),Hollyn,I was wrong (Wrong) to ever let you go You're ...,43,5RoaRB24uoKbClziJyISOo,Everything and More / Isaac,2018-10-19,Christian Dance Party,37i9dQZF1DWUUPO0Sbx2CM,pop,dance pop,0.656,0.700,4,-7.176,0,0.0431,0.24900,0.000000,0.2210,0.293,100.010,197000,en,Fear,0,0,0


In [ ]:
from google.colab import files
kaggle_data_v2.to_csv('audio_features_predictions.csv')
files.download('audio_features_predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# PREPROCESSING
kaggle_data = kaggle_data.drop_duplicates()

def lemmatize(text):
  lemmatizer = WordNetLemmatizer()
  tokenization = nltk.word_tokenize(text)
  return ' '.join([lemmatizer.lemmatize(words) for words in tokenization ])

def strip_punct(text):
  table = str.maketrans(dict.fromkeys(string.punctuation))
  return text.translate(table)

def strip_stop(unwanted_word_list, text):
  return ' '.join(['' if word in unwanted_word_list else word for word in text.split()])

#convert to lowercase
spotify_songs["lyrics"] = spotify_songs["lyrics"].str.lower()
kaggle_data["lyrics"] = kaggle_data["lyrics"].str.lower()

# Strip punctuation
spotify_songs["lyrics"] = spotify_songs["lyrics"].apply(lambda x: strip_punct(x))
kaggle_data["lyrics"] = kaggle_data["lyrics"].apply(lambda x: strip_punct(x))

# # Lemmatize all the words
stemmer = WordNetLemmatizer()
spotify_songs["lyrics"] = spotify_songs["lyrics"].apply(lambda x: lemmatize(x))
kaggle_data["lyrics"] = kaggle_data["lyrics"].apply(lambda x: lemmatize(x))

# # Strip stop words
stop_words = stopwords.words('english')
spotify_songs["lyrics"] = spotify_songs["lyrics"].apply(lambda x: strip_stop(stop_words, x))
kaggle_data["lyrics"] = kaggle_data["lyrics"].apply(lambda x: strip_stop(stop_words, x))

In [ ]:
# [START] VERSION 3: COMBINED AUDIO FEATURES + BOW LYRICS
from sklearn_pandas import DataFrameMapper
kaggle_data_v3 = kaggle_data.copy()
x_combo_set = spotify_songs[['danceability', 'energy', 'loudness', 'valence', 'lyrics']]
y_combo_set = spotify_songs['isHappy']

mapper = DataFrameMapper([
    (['danceability', 'energy', 'loudness', 'valence'], None),
    ('lyrics', CountVectorizer(binary=True, min_df=5))
])
combined_X = mapper.fit_transform(x_combo_set)
print('combined_X shape:', combined_X.shape)
combo_model = LogisticRegression(penalty='l2').fit(combined_X, y_combo_set)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


combined_X shape: (5686, 33589)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
combined_X_kaggle = mapper.transform(kaggle_data_v3[['danceability', 'energy', 'loudness', 'valence', 'lyrics']])
print('combined_X_kaggle shape:', combined_X_kaggle.shape)

combo_preds = combo_model.predict(combined_X_kaggle)
kaggle_data_v3['combo_features_preds'] = combo_preds
kaggle_data_v3

In [ ]:
from google.colab import files
kaggle_data_v3.to_csv('combo_lyrics_features_predictions.csv')
files.download('combo_lyrics_features_predictions.csv')
# [END] VERSION 3: COMBINED AUDIO FEATURES + BOW LYRICS

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Bag of words model happy
vectorizer = CountVectorizer(binary=True, min_df=5)
happy_spot_x = vectorizer.fit_transform(spotify_songs['lyrics'])

happy_spot_x.toarray()
happy_spot_model = LogisticRegression(penalty='l2').fit(happy_spot_x, spotify_songs['isHappy'])
y_train_pred = happy_spot_model.predict(happy_spot_x)
train_f1_score = f1_score(spotify_songs['isHappy'], y_train_pred)
train_f1_score




/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.9839972761321076

In [ ]:
# Test happy 
test = vectorizer.transform(kaggle_data['lyrics'])
y_test_pred = happy_spot_model.predict(test)
test_f1_score = f1_score(kaggle_data['isHappy'], y_test_pred)
test_f1_score



0.3378160301237224

In [ ]:
kaggle_data['isHappy_pred'] = y_test_pred

In [ ]:
#Bag of words model sad
vectorizer = CountVectorizer(binary=True, min_df=5)
sad_spot_x = vectorizer.fit_transform(spotify_songs['lyrics'])

sad_spot_x.toarray()
sad_spot_model = LogisticRegression(penalty='l2').fit(sad_spot_x, spotify_songs['isSad'])
y_train_pred = sad_spot_model.predict(sad_spot_x)
train_f1_score = f1_score(spotify_songs['isSad'], y_train_pred)
train_f1_score



In [ ]:
# Test sad
test = vectorizer.transform(kaggle_data['lyrics'])
y_test_pred = sad_spot_model.predict(test)
test_f1_score = f1_score(kaggle_data['isSad'], y_test_pred)
test_f1_score

In [ ]:
kaggle_data['isSad_pred'] = y_test_pred

In [ ]:
kaggle_data

In [ ]:
kaggle_data.to_csv('spotify_songs_predictions.csv')
files.download('spotify_songs_predictions.csv')

In [ ]:
#################################combine datasets###############################

In [ ]:
#combine datasets

#kaggle random data 
kaggle_test = raw_kaggle_songs.sample(n=10000, random_state=1).reset_index(drop=True)
kaggle_train = pd.concat([raw_kaggle_songs, kaggle_test]).drop_duplicates(keep=False).sample(n=5000, random_state=1).reset_index(drop=True)

spotify_train = spotify_songs.sample(n=5000, random_state=1).reset_index(drop=True)
combo_data = pd.concat([kaggle_train, spotify_train]).drop_duplicates(keep=False).reset_index(drop=True)

combo_data['isHappy'] = combo_data.apply(lambda row: 1 if row['label'] == 'Joy' else 0, axis=1)
combo_data['isSad'] = combo_data.apply(lambda row: 1 if row['label'] == 'Sadness' else 0, axis=1)

kaggle_test['isHappy'] = kaggle_test.apply(lambda row: 1 if row['label'] == 'Joy' else 0, axis=1)
kaggle_test['isSad'] = kaggle_test.apply(lambda row: 1 if row['label'] == 'Sadness' else 0, axis=1)


combo_data

,Unnamed: 0,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language,label,artist,album,time_signature,isHappy,isSad
0,11879,524wRHQekJJvQWtAIy115v,Where Are You Tonight,Coro,Here I lay all alone in the night Beside mysel...,38.0,7E4FS3eSJtdjI00NBbtnDY,Freestyle's Best Extended Versions Volumes 1 & 2,2003,80's Freestyle/Disco Dance Party (Set Crossfad...,1oReEujyWpQv2OX68BVPPA,latin,latin hip hop,0.715,0.572,6,-10.331,0,0.0586,0.00522,0.102000,0.0850,0.6700,118.343,313507,en,Sadness,NaN,NaN,NaN,0,1
1,11891,52dm9op3rbfAkc1LGXgipW,Wild Horses,The Rolling Stones,Childhood living is easy to do The things you ...,72.0,29m6DinzdaD0OPqWKGyMdz,Sticky Fingers (Remastered),1971-04-23,Rock Classics,37i9dQZF1DWXRqgorJj26U,rock,classic rock,0.432,0.389,7,-6.517,1,0.0255,0.68700,0.010700,0.0788,0.1580,139.506,341773,en,Sadness,NaN,NaN,NaN,0,1
2,12334,5byYO8JjkxV27yQYo2pDRx,Summergirls,Dino,Summertime's in the air And summergirls are ev...,28.0,6CC4LpUaizMwq7yOzSpwNo,24/7,1989-01-01,80's Freestyle/Disco Dance Party (Set Crossfad...,1oReEujyWpQv2OX68BVPPA,latin,latin hip hop,0.764,0.580,9,-15.200,0,0.0930,0.00219,0.055100,0.1460,0.8100,121.343,376573,en,Joy,NaN,NaN,NaN,1,0
3,739,0eXU92qkEfIWu4SQOILFU3,Still Got Me (feat. Jocelyn Bowman),GAWVI,"Is it your touch giving me life, oh It's such ...",0.0,6ZyLjp2vCP3qUeII4Diu64,Holding Hue - EP,2016-09-09,CHRISTIAN ELECTRO / DANCE / EDM,0MhTMIo1bgH6zzPh7BdChT,edm,progressive electro house,0.658,0.634,6,-6.256,1,0.0373,0.09290,0.004030,0.1080,0.6530,112.053,230364,en,Joy,NaN,NaN,NaN,1,0
4,4023,1PwxKTZydKhqiMHRZoFfn4,Step Up,Pete Rock / InI,"A little bit, if you can get it out I ain't go...",0.0,4VVJkmbxlv2y8xkeaSWbsE,Hip Hop Underground Soul Classics,2003-11-04,Smooth Hip Hop,7ozFIvz1bE1HPgQTUl5qfT,r&b,hip pop,0.778,0.695,1,-8.662,1,0.2420,0.29000,0.000347,0.1080,0.6640,90.925,285200,en,Fear,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,3641,0HuNPrWCyZpCx59pu9ON99,Immortality (feat. Bee Gees),NaN,So this is who I am\nAnd this is all I know\nA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.546,0.315,2,-11.208,1,0.0324,NaN,0.000003,0.0795,0.0756,82.951,250893,NaN,NaN,Céline Dion,Let's Talk About Love,4.0,0,0
9996,701,0Ph6L4l8dYUuXFmb71Ajnd,Livin' la Vida Loca,NaN,[Verse 1]\nShe's into superstitions\nBlack cat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.425,0.954,1,-3.756,0,0.0476,NaN,0.000000,0.0555,0.9330,178.043,243160,NaN,NaN,Ricky Martin,Ricky Martin,4.0,0,0
9997,2928,3S4px9f4lceWdKf0gWciFu,Cheap Thrills,NaN,"[Verse 1]\nCome on, come on, turn the radio on...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.628,0.698,6,-5.608,0,0.1050,NaN,0.001430,0.0907,0.7320,89.976,211667,NaN,NaN,Sia,This Is Acting (Deluxe Version),4.0,0,0
9998,5031,60UpYVsYj2ytkFay5Rgeg1,Love You All over Again,NaN,There are a lot of things in this life that do...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.528,0.759,2,-4.874,1,0.0264,NaN,0.000000,0.1100,0.5470,89.927,212507,NaN,NaN,"Love, Lies and Fiction",Love You All over Again,4.0,0,0


In [ ]:
# PREPROCESSING

def lemmatize(text):
  lemmatizer = WordNetLemmatizer()
  tokenization = nltk.word_tokenize(text)
  return ' '.join([lemmatizer.lemmatize(words) for words in tokenization ])

def strip_punct(text):
  table = str.maketrans(dict.fromkeys(string.punctuation))
  return text.translate(table)

def strip_stop(unwanted_word_list, text):
  return ' '.join(['' if word in unwanted_word_list else word for word in text.split()])

#convert to lowercase
combo_data["lyrics"] = combo_data["lyrics"].str.lower()
kaggle_test["lyrics"] = kaggle_test["lyrics"].str.lower()

# Strip punctuation
combo_data["lyrics"] = combo_data["lyrics"].apply(lambda x: strip_punct(x))
kaggle_test["lyrics"] = kaggle_test["lyrics"].apply(lambda x: strip_punct(x))

# # Lemmatize all the words
stemmer = WordNetLemmatizer()
combo_data["lyrics"] = combo_data["lyrics"].apply(lambda x: lemmatize(x))
kaggle_test["lyrics"] = kaggle_test["lyrics"].apply(lambda x: lemmatize(x))

# # Strip stop words
stop_words = stopwords.words('english')
combo_data["lyrics"] = combo_data["lyrics"].apply(lambda x: strip_stop(stop_words, x))
kaggle_test["lyrics"] = kaggle_test["lyrics"].apply(lambda x: strip_stop(stop_words, x))

In [ ]:
#Bag of words model combo -- sad 
vectorizer = CountVectorizer(binary=True, min_df=5)
combo_x = vectorizer.fit_transform(combo_data['lyrics'])

combo_x.toarray()
combo_model = LogisticRegression(penalty='l2').fit(combo_x, combo_data['isSad'])
y_train_pred =  combo_model.predict(combo_x)
train_f1_score = f1_score(combo_data['isSad'], y_train_pred)
train_f1_score




/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.9867976665643231

In [ ]:
# Test combo -- sad
test = vectorizer.transform(kaggle_test['lyrics'])
y_test_pred = combo_model.predict(test)
test_f1_score = f1_score(kaggle_test['isSad'], y_test_pred)
test_f1_score

0.6438063986874487

In [ ]:
combo_data['isSad_pred'] = y_test_pred
combo_data

In [ ]:
#Bag of words model combo -- happy 
vectorizer = CountVectorizer(binary=True, min_df=5)
combo_x = vectorizer.fit_transform(combo_data['lyrics'])

combo_x.toarray()
combo_model = LogisticRegression(penalty='l2').fit(combo_x, combo_data['isHappy'])
y_train_pred =  combo_model.predict(combo_x)
train_f1_score = f1_score(combo_data['isHappy'], y_train_pred)
train_f1_score


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.9851603638104356

In [ ]:
# Test combo -- happy
test = vectorizer.transform(kaggle_test['lyrics'])
y_test_pred = combo_model.predict(test)
test_f1_score = f1_score(kaggle_test['isHappy'], y_test_pred)
test_f1_score

0.6340519624101713

In [ ]:
combo_data['isHappy_pred'] = y_test_pred

In [ ]:
combo_data

In [ ]:
#making outputted happy song dataset 
happy_songs = pd.DataFrame()

for index, row in combo_data.iterrows():
    if combo_data['isHappy_pred'][index] == 1:
        #print(combo_data.iloc[index])
        happy_songs = happy_songs.append(combo_data.iloc[index], ignore_index = True)

In [ ]:
happy_songs

In [ ]:
#making outputted sad song dataset 
sad_songs = pd.DataFrame()

for index, row in combo_data.iterrows():
    if combo_data['isSad_pred'][index] == 1:
        #print(combo_data.iloc[index])
        sad_songs = sad_songs.append(combo_data.iloc[index], ignore_index = True)

In [ ]:
sad_songs

In [ ]:
from google.colab import files
happy_songs.to_csv('outputted_happy_songs.csv')
files.download('outputted_happy_songs.csv')

In [ ]:
sad_songs.to_csv('outputted_sad_songs.csv')
files.download('outputted_sad_songs.csv')